# Weitere Beispiele zur Poisson-Gleichung
Nachdem wir im [vorherigen Beispiel](./tutorial-fem-01.ipynb) die wesentlichen  Schritte zur Diskretisierung der Poisson-Gleichung kennengelernt haben, wollen wir nun die Gleichung
$$\nabla \cdot[-D \nabla u] = f $$
mit Diffusionskonstante $D$ und Quelle $f$ in verschiedenen Beispiele näher untersuchen. U.a. werden wir den Diskretisierungsfehler bestimmen.



## Problemdefinitionen

Die Problemdefinitionen legen wir nun in verschiedene LUA-Tabellen ab:
### Problem 1

Die folgende Tabelle kodiert das [vorherige Beispiel](./tutorial-fem-01.ipynb) zur Laplace-Gleichung ($D=1$) und einer rechten Seite $f=0$ in Kurzform:

In [45]:
SQUARE_CONFIG =
{
    -- Geometrie
    gridName= "grids/unit_square_tri.ugx", --"grids/unit_square_tri.ugx",
    requiredSubsets = {"Inner", "Boundary"},
    numRefs= 2,
    
    -- Randbedingungen
    dirichletData = 
    {
        subsets = "Boundary", callback = "MyDirichletBndCallback",  
    },
    
    -- Parameter fuer PDE    
    diffusion = 1.0,  -- D=1.0
    source = 0.0,    -- f=0
}

-- Callback fuer Randbedingungen
function MyDirichletBndCallback(x, y, t)
    if (y==1) then 	return true, 0.0 
    elseif (y==0) then  return true, math.sin(math.pi*1*x)
    else return false, 0.0 
    end
end


### Problem 2

Im folgenden Beispiel ist wieder $D=1$. Die rechte Seite $f$ wird so gesetzt, dass sich Sinusschwingung 

$$u(x,y) = \sin (\mu  \pi x) \sin (\nu  \pi y)$$

als Lösung ergibt. Außerdem diskretisieren wir das Einheitsquadrat mit Vierecken:

In [46]:
SQUARE_CONFIG_WITH_RHS =
{
    -- Geometrie
    gridName= "grids/unit_square_quad.ugx",
    requiredSubsets = {"Inner", "Boundary"},
    numRefs= 3,
    
    -- Randbedingungen
    dirichletData = 
    {
        subsets = "Boundary", callback = "C2_MyDirichletBndCallback",  
    },
    
    diffusion = 1.0,
    source = "C2_MySourceCallback", 
    myref = "C2_MyRefCallback",
}

-- Callback fuer Randbedingungen
function C2_MyDirichletBndCallback(x, y, t)
     return true, 0.0 
end

-- Callback fuer rechte Seite
function C2_MySourceCallback(x, y, t)
    local mu = 1.0
    local nu = 4.0
    local scale =  (mu*mu + nu*nu)*(math.pi)*(math.pi)
    return scale*math.sin(math.pi*mu*x)* math.sin(math.pi*nu*y)
end

-- Callback fuer Referenz
function C2_MyRefCallback(x, y, t)
    local mu = 1.0
    local nu = 4.0
    return math.sin(math.pi*mu*x)* math.sin(math.pi*nu*y)
end



### Problem 3

In [47]:
SECTOR_CONFIG =
{
    gridName= "grids/sectorTest.ugx",
    requiredSubsets = {"Inner", "Circle", "Cut"},
    numRefs= 2,
    
    dirichletData = 
    {
        subsets = "Circle, Cut", callback = "SectorDirichletSol",  
    },
    
    diffusion = 1.0,
    source = 0.0, 
    myref = "SectorDirichletSol"
}

-- callback function boundary values (only the ones matching 'dim' are used)
function SectorDirichletSol(x, y, t, si)
    local r = math.sqrt(x*x+y*y);
    local phi = math.atan2(y,x);
    if (phi<0) then phi = phi + 2*math.pi; end
    val=math.pow(r,(2/3))*math.sin(phi/3.0*2);
    return val
end

### Auswahl einer Konfiguration
Aus den o.g. Konfigurationen wählen wir eine aus:

In [48]:
CONFIG = SQUARE_CONFIG_WITH_RHS

## Konfiguration von UG4

Die folgenden Schritte sind aus dem [vorherigen Beispiel](./tutorial-fem-01.ipynb) bekannt:

### Initialisierung 

In [ ]:
InitUG(2, AlgebraType("CPU", 1));  -- Initialize world dimension dim=2 and default algebra type
ug_load_script("ug_util.lua")           -- Load utility scripts (e.g. from from ugcore/scripts)
ug_load_script("util/refinement_util.lua")

### Einlesen des Rechengebietes aus einer Datei

In [49]:
dom = Domain()
LoadDomain(dom, CONFIG.gridName)
dom = util.CreateDomain(CONFIG.gridName, CONFIG.numRefs, CONFIG.requiredSubsets)

Loading Domain grids/unit_square_quad.ugx ... done.
Performing integrity check on domain ... done.
Refining(3): 1 2 3 done.


### Ansatzraum

In [50]:
-- Setup for FEM approximation space.
approxSpace = ApproximationSpace(dom)
approxSpace:add_fct("u", "Lagrange", 1)  -- Linear ansatz functions

-- More inits.
approxSpace:init_levels()
approxSpace:init_top_surface()
approxSpace:print_statistic()

| ---------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                  |
|  Algebra: Block 1 (divide by 1 for #Index)                                   |
|                                                                              |
|    GridLevel   |       Domain |     0: Inner |  1: Boundary                  |
| ---------------------------------------------------------------------------- |
| (lev,    0)    |            9 |            1 |            8 |
| (lev,    1)    |           25 |            9 |           16 |
| (lev,    2)    |           81 |           49 |           32 |
| (lev,    3)    |          289 |          225 |           64 |
| (lev,    0, g) |            9 |            1 |            8 |
| (lev,    1, g) |           25 |            9 |           16 |
| (lev,    2, g) |           81 |           49 |           32 |
| (lev,    3, g) |          289 |          225 |           64 |
| 

### Diskretisierung

Erzeuge Objekt für eine **Elementdiskretisierung** für die Konvektions-Diffusionsgleichung.

In [51]:
elemDisc = ConvectionDiffusion("u", "Inner", "fe")
elemDisc:set_diffusion(CONFIG.diffusion)

if (CONFIG.source) then
    elemDisc:set_source(CONFIG.source)
end

Erzeuge Objekt für **Randbedingungen**:

In [52]:
dirichletBND = DirichletBoundary()
dirichletBND:add(CONFIG.dirichletData.callback, "u", CONFIG.dirichletData.subsets)

Füge beides zu einer Gebietsdiskretisierung hinzu:

In [53]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc)
domainDisc:add(dirichletBND)

### Konfiguration eines iterativen Lösers

Ein Mehrgitterverfahren hat lediglich lineare Komplexität

In [54]:
-- set up solver (using 'util/solver_util.lua')
local solverDesc = {
    type = "bicgstab",
    precond = {
        type = "gmg",
        approxSpace = approxSpace,
        smoother = "sgs",
        baseSolver = "lu"
    }
}
solver = util.solver.CreateSolver(solverDesc)

### Assembliere und löse LGS


In [55]:
Ah = AssembledLinearOperator(domainDisc)
uh = GridFunction(approxSpace)
bh = GridFunction(approxSpace)
uh:set(0.0)


domainDisc:assemble_linear(Ah, bh)
domainDisc:adjust_solution(uh)

solver:init(Ah, uh)
solver:apply(uh, bh)


   % %%%%%%%%             BiCGStab              %%%%%%%%%%%
   % %%%%%%%%   (Precond: Geometric MultiGrid)  %%%%%%%%%%%
   %   Iter      Defect         Rate 
   %    0:    4.962649e+00      -------
   %    1:    2.027937e-02    4.086399e-03
   %    2:    1.862548e-05    9.184448e-04
   %    3:    3.746602e-07    2.011547e-02
   % Relative reduction 1.000000e-06 reached after 3 steps.
   % Average reduction over 3 steps: 4.226439e-03
   % %%%%%  Iteration converged  %%%%%



Ausgabe als vtk bzw. vec-Datei.

In [25]:
local solFileName = "u_solution"
WriteGridFunctionToVTK(uh, solFileName)

## Fehleranalyse

Falls die  Lösung analytisch bekannt ist, können wir den Diskretisierungsfehler der numerischen Lösung $u_h$ bestimmen.  

Dies geschieht in der L2-Norm: $$\|u-u_h\|_0 := \sqrt{\int_\Omega (u-u_h)^2 }$$

In [26]:
if (CONFIG.myref) then
    err0=L2Error(CONFIG.myref,  uh, "u", 1.0, 4)
    print(CONFIG.numRefs)
    print(err0)
end

3
0.026669178876216


 Alternativ kann die H1-Norm verwendet werden, welche auch Ableitungen berücksichtigt: $$\|u-u_h\|_1 := \sqrt{\int_\Omega (u-u_h)^2+ (\nabla (u-u_h))^2 }$$


In [39]:
if (CONFIG.myref) then
    uref = uh:clone()
    Interpolate(CONFIG.myref, uref, "u")
    err1=H1Error(uref, "c",  uh, "u", 1.0, "Inner")
    print(err1)
end

LUA-ERROR: ---


 % EXCEPTION on Proc 0: UGError thrown
 %   Error traceback (innermost first): 
 %   0: Function name u not found in pattern.
 %      [at ...git/ugcore/ugbase/lib_disc/dof_manager/function_pattern.cpp:136]
 % In CALL to function 'Interpolate(string LuaFunction, SmartPtr<GridFunction2dCPU1> GridFunction, string Component)'
 %    ARGS: (cstring "C2_MyRefCallback", SmartPtr<GridFunction2dCPU1> 0x7fcac287d7e0, cstring "u")
 % In FILE: buffer:3
 % At LINE: ''
LUA-ERROR! Call stack:
   1  buffer:3        
 % ABORTING script parsing.


Die Werte speichern wir in einer Tabelle:

In [40]:
esquare={
    -- numRefs, err0, err1
    {3, 0.026669178876216,0.036854207999089 },
    {4, 0.0067063542743748,0.0096692527695234},
    {5, 0.0016790153921112,0.0024457630567399},
    {6, 0.00041990545869718,0.00061321594052763},
    {7, 0.0001049859512669,0.0001534133061914},
}

In [41]:
esector={
    {3, 0.0021517562195368, 0.014347178579376 },
    {4, 0.00082670975412983, 0.0090798283058054},
    {5, 0.00032006823581697, 0.0057306863871163},
    {6, 0.00013039000095758, 0.003629782236302},
    {7, 5.5487900630332e-05, 0.0022953491311963},
}

Mit folgender Funktion lassen sich  die Werte in Gnuplot darstellen und in eine Datei schreiben:

In [42]:
function VisualizeGnuplot(etable, title, filename)

    local file = io.open("results.txt", "w") -- opens a file in write mode
    io.output(file) 
    for i,entry in ipairs(etable) do
        print(entry[1].."\t"..entry[2].."\t"..entry[3].."\n")
        io.write(entry[1].."\t"..entry[2].."\t"..entry[3].."\n")
    end
    io.close(file)
    
    local file = io.open("cmd.gnuplot", "w") -- opens a file in write mode
    io.output(file)
   
    io.write("set xlabel \"#Verfeinerungen\";\n" )
    io.write("set ylabel \"Fehler\";\n" )
    io.write("set terminal png;\n")
    io.write("set output \""..filename.."\";\n")
    io.write("set logscale y;\n")
    io.write("plot \"results.txt\" using 1:2 title \""..title.."\"with linespoints;\n")
    io.close(file)  
    
    os.execute('gnuplot <  cmd.gnuplot')
end

Diese Funktion rufen wir nun auf und stellen das Ergebnis dar (ggf. muss die Grafik wegen Caching im Browser explizit neu gelanden werden!):

In [43]:
VisualizeGnuplot(esquare, "Error - Square", "esquare.png")

3	0.026669178876216	0.036854207999089

4	0.0067063542743748	0.0096692527695234

5	0.0016790153921112	0.0024457630567399

6	0.00041990545869718	0.00061321594052763

7	0.0001049859512669	0.0001534133061914



![](esquare.png)